In [ ]:
!pip install rank_bm25 kiwipiepy sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 53.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 104.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=34ed71ab41ad8bba54ff12fb22c05f5c1de10f1f294976482887324ccf980e56
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc8

In [ ]:
!wget -c https://github.com/euphoris/datasets/raw/master/science_books.csv

--2023-04-11 06:36:59--  https://github.com/euphoris/datasets/raw/master/science_books.csv
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/euphoris/datasets/master/science_books.csv [following]
--2023-04-11 06:36:59--  https://raw.githubusercontent.com/euphoris/datasets/master/science_books.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89217 (87K) [text/plain]
Saving to: ‘science_books.csv’

science_books.csv   100%[===================>]  87.13K  --.-KB/s    in 0.007s  

2023-04-11 06:37:00 (11.9 MB/s) - ‘science_books.csv’ saved [89217/89217]



In [ ]:
import pandas as pd
books = pd.read_csv('science_books.csv')

In [ ]:
books.head()

,제목
0,"물고기는 존재하지 않는다 : 상실, 사랑 그리고 숨어 있는 삶의 질서에 관한 이야기"
1,코스모스 : 특별판
2,이기적 유전자 The Selfish Gene
3,세상은 실제로 어떻게 돌아가는가 : 우리의 문명을 정확하게 이해하기 위한 과학적 접근
4,"[예스리커버] 정재승의 과학콘서트 : 복잡한 세상 명쾌한 과학(개정증보2판, 양장)"


In [ ]:
from kiwipiepy import Kiwi
kiwi = Kiwi()

In [ ]:
def tokenize(sent):
    for token in kiwi.tokenize(sent):
        if token.tag in {'NNG', 'NNP', 'SL', 'VV', 'VA'}:
            yield token.form, token.tag

In [ ]:
list(tokenize('물고기는 존재하지 않는다 : 상실, 사랑 그리고 숨어 있는 삶의 질서에 관한 이야기'))

[('물고기', 'NNG'),
 ('존재', 'NNG'),
 ('상실', 'NNG'),
 ('사랑', 'NNG'),
 ('그리', 'VV'),
 ('숨', 'VV'),
 ('삶', 'NNG'),
 ('질서', 'NNG'),
 ('관하', 'VV'),
 ('이야기', 'NNG')]

In [ ]:
tokenized_corpus = []
for title in books.제목:
    tokenized_corpus.append(list(tokenize(title)))

In [ ]:
tokenized_corpus[2]

[('이기', 'NNG'),
 ('유전자', 'NNG'),
 ('The', 'SL'),
 ('Selfish', 'SL'),
 ('Gene', 'SL')]

In [ ]:
from rank_bm25 import BM25Okapi
bm25 = BM25Okapi(tokenized_corpus)

In [ ]:
idf_table = pd.DataFrame(bm25.idf.items(), columns=['token', 'idf'])

In [ ]:
idf_table.sort_values('idf')

,token,idf
24,"(과학, NNG)",1.590378
169,"(수학, NNG)",2.076635
9,"(이야기, NNG)",2.226424
17,"(세상, NNG)",2.396806
31,"(양장, NNG)",2.396806
...,...,...
1249,"(아이디어, NNG)",6.501790
1248,"(보듬, VV)",6.501790
1247,"(이웃, NNG)",6.501790
1621,"(스타일링, NNG)",6.501790


In [ ]:
query = list(tokenize('다정한 것이 살아남는다'))

In [ ]:
bm25.get_top_n(query, books.제목, n=5)

['다정한 것이 살아남는다 : 친화력으로 세상을 바꾸는 인류의 진화에 관하여(10만부 기념 스페셜 에디션, 저자 친필 사인 인쇄본)',
 '낙타는 왜 사막으로 갔을까 : 살아남은 동물들의 비밀',
 '무엇이 우리를 다정하게 만드는가 : 타인을 도우려 하는 인간 심리의 뇌과학적 비밀(양장)',
 '우주에서 기다릴게 : 한국 첫 우주인이 펼치는 다정한 호기심의 기록',
 '다정한 물리학 : 거대한 우주와 물질의 기원을 탐구하고 싶을 때']

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
sbert = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

In [ ]:
texts = ['졸음이 쏟아지다', '피곤하다', '비가 쏟아지다']
emb = sbert.encode(texts)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_similarity(emb)

array([[1.0000001 , 0.61894107, 0.33336684],
       [0.61894107, 1.0000005 , 0.21717583],
       [0.33336684, 0.21717583, 1.        ]], dtype=float32)

In [ ]:
emb = sbert.encode(books.제목)

In [ ]:
query_emb = sbert.encode(['다정한 것이 살아남는다'])

In [ ]:
sims = cosine_similarity(query_emb, emb)

In [ ]:
sims.shape

(1, 1000)

In [ ]:
import numpy as np
books.iloc[np.argsort(-sims[0])[:5]]

,제목
322,"다정함의 과학 : 친절, 신뢰, 공감 속에 숨어 있는 건강과 행복의 비밀"
6,다정한 것이 살아남는다 : 친화력으로 세상을 바꾸는 인류의 진화에 관하여(10만부 ...
618,모든 것은 그 자리에 : 첫사랑부터 마지막 이야기까지(양장)
392,이토록 다정한 기술 : 지구와 이웃을 보듬는 아이디어(〈희망의 이유〉 사쉐 증정 (...
111,무엇이 우리를 다정하게 만드는가 : 타인을 도우려 하는 인간 심리의 뇌과학적 비밀(양장)


In [ ]:
norm_emb = emb / np.linalg.norm(emb, axis=1, keepdims=True)

In [ ]:
(emb[1] ** 2).sum()

356.24344

In [ ]:
(norm_emb[1] ** 2).sum()

1.0

In [ ]:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
clustering = AgglomerativeClustering(n_clusters=10)
cluster_labels = clustering.fit_predict(norm_emb)

In [ ]:
books.제목[cluster_labels == 1]

1                                        코스모스 : 특별판
13                              종의 기원(양장)-드디어 다윈 01
15               사이코패스 뇌과학자 : 괴물은 태어나는가, 만들어지는가(양장)
19     미적분의 쓸모 : 보통 사람들도 이해하는 새로운 미래의 언어(양장, 개정증보판)
24      나는 풍요로웠고, 지구는 달라졌다(〈희망의 이유〉 사쉐 증정 (포인트 차감))
                           ...                     
965                                  6일만에 끝내는 미분방정식
968                                 고생물도감 고생대 편(양장)
981                          산야초 대사전 : 내 몸을 살리는(양장)
982                     정글  : 포티큘러 북(양장)-포티큘러 북 002
990    두 얼굴의 백신(포함 국내도서 2만원↑ 〈카를로 로벨리 맥주잔〉(포인트 차감))
Name: 제목, Length: 131, dtype: object

In [ ]:
categories = ['요금 문제', '불량', '직원 태도', '상품', '회원']

In [ ]:
descriptions = [
    '청구 오류, 불명확한 요금 설명, 불필요한 부과 금액',
    '사용 불가, 품질 불량',
    '불친절, 무관심, 불친절한 대응',
    '부실한 상품 설명, 불만족스러운 상품 수준',
    '가입 문의, 해지/변경 처리 오류'
]

In [ ]:
desc_emb = sbert.encode(descriptions)

In [ ]:
complaint = [
    '저번 달 청구서에는 없던 부과금이 추가로 청구되어 불만입니다.',
    '제품이 작동이 안되요, 다시 연결을 시도해도 똑같은 문제가 발생합니다.',
    '전화상으로 요금제 변경을 신청했는데, 왜 아직도 변경되지 않은 건가요?'
]

In [ ]:
comp_emb = sbert.encode(complaint)

In [ ]:
sims = cosine_similarity(comp_emb, desc_emb)

In [ ]:
for i, text in enumerate(complaint):
    cat = categories[np.argmax(sims[i])]
    print(f'{text} [{cat}]')

저번 달 청구서에는 없던 부과금이 추가로 청구되어 불만입니다. [요금 문제]
제품이 작동이 안되요, 다시 연결을 시도해도 똑같은 문제가 발생합니다. [불량]
전화상으로 요금제 변경을 신청했는데, 왜 아직도 변경되지 않은 건가요? [회원]
